In [38]:
import pandas as pd
import os

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorIndexer

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from datetime import datetime
from pyspark.sql.functions import col,udf
from pyspark.sql.types import DateType
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

In [2]:
sample1 = pd.read_csv("Training_Data_Nov9(NEW).csv")

In [18]:
sample2 = pd.read_csv("validation_df_Nov9.csv")

In [19]:
sample1

,Unnamed: 0,VendorID_1_0,store_and_fwd_flag_1_0,RateCode_stdrate,RateCode_JFK,RateCode_Newark,RateCode_Nassau_W,RateCode_Negotiated,RateCode_Group,Pickup_Manhattan,...,pickup_dayofweek,pickup_weekday_name,pickup_hour,pickup_month,ptype_CCard,ptype_cash,ptype_nocharge,ptype_dispute,ptype_voidtrip,Duration
0,0,0,0,0,0,0,0,1,0,0,...,6,Sunday,20,7,0,1,0,0,0,2230
1,1,0,0,0,0,0,0,1,0,1,...,4,Friday,0,10,0,1,0,0,0,245
2,2,0,0,0,0,0,0,1,0,1,...,1,Tuesday,2,9,1,0,0,0,0,105
3,3,0,0,1,0,0,0,0,0,0,...,0,Monday,8,3,0,1,0,0,0,495
4,4,0,0,1,0,0,0,0,0,0,...,0,Monday,17,2,1,0,0,0,0,1309
5,5,0,0,1,0,0,0,0,0,0,...,5,Saturday,18,2,0,1,0,0,0,536
6,6,0,0,1,0,0,0,0,0,0,...,2,Wednesday,8,2,0,1,0,0,0,387
7,7,0,0,1,0,0,0,0,0,0,...,5,Saturday,3,3,1,0,0,0,0,766
8,8,0,0,1,0,0,0,0,0,0,...,5,Saturday,23,3,0,1,0,0,0,688
9,9,0,0,1,0,0,0,0,0,0,...,3,Thursday,19,1,1,0,0,0,0,276


In [20]:
sample1.shape

(36065, 51)

In [21]:
sample1.columns

Index(['Unnamed: 0', 'VendorID_1_0', 'store_and_fwd_flag_1_0',
       'RateCode_stdrate', 'RateCode_JFK', 'RateCode_Newark',
       'RateCode_Nassau_W', 'RateCode_Negotiated', 'RateCode_Group',
       'Pickup_Manhattan', 'Pickup_Brooklyn', 'Pickup_Queens', 'Pickup_Bronx',
       'DropOff_Manhattan', 'DropOff_Brooklyn', 'DropOff_Queens',
       'DropOff_Bronx', 'avg_temp_C', 'Rain', 'Fog', 'Snow', 'Holiday',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'pickup_zip', 'pickup_neighborhood', 'dropoff_zip',
       'dropoff_neighborhood', 'pickup_dayofweek', 'pickup_weekday_name',
       'pickup_hour', 'pickup_month', 'ptype_CCard', 'ptype_cash',
       'ptype_nocharge', 'ptype_dispute', 'ptype_voidtrip', 'Duration'],
      dtype='object')

In [3]:
sample1 = sample1.drop(['Unnamed: 0'], axis=1)

In [4]:
samdf = spark.createDataFrame(sample1)

In [19]:
valdf = spark.createDataFrame(sample2)

In [25]:
samdf.show(5)

+------------+----------------------+----------------+------------+---------------+-----------------+-------------------+--------------+----------------+---------------+-------------+------------+-----------------+----------------+--------------+-------------+----------+----+---+----+-------+-------------------+-------------------+---------------+-------------+------------------+------------------+------------------+------------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------------------+-----------+--------------------+----------------+-------------------+-----------+------------+-----------+----------+--------------+-------------+--------------+--------+
|VendorID_1_0|store_and_fwd_flag_1_0|RateCode_stdrate|RateCode_JFK|RateCode_Newark|RateCode_Nassau_W|RateCode_Negotiated|RateCode_Group|Pickup_Manhattan|Pickup_Brooklyn|Pickup_Queens|Pickup_Bronx|DropOff_Manhattan|DropOff_Brooklyn|DropOff_Queens|DropOff_Bronx|avg_temp_C|R

In [20]:
valdf.columns

['Unnamed: 0',
 'VendorID_1_0',
 'store_and_fwd_flag_1_0',
 'RateCode_stdrate',
 'RateCode_JFK',
 'RateCode_Newark',
 'RateCode_Nassau_W',
 'RateCode_Negotiated',
 'RateCode_Group',
 'Pickup_Manhattan',
 'Pickup_Brooklyn',
 'Pickup_Queens',
 'Pickup_Bronx',
 'DropOff_Manhattan',
 'DropOff_Brooklyn',
 'DropOff_Queens',
 'DropOff_Bronx',
 'avg_temp_C',
 'Rain',
 'Fog',
 'Snow',
 'Holiday',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'pickup_zip',
 'pickup_neighborhood',
 'dropoff_zip',
 'dropoff_neighborhood',
 'pickup_dayofweek',
 'pickup_weekday_name',
 'pickup_hour',
 'pickup_month',
 'ptype_CCard',
 'ptype_cash',
 'ptype_nocharge',
 'ptype_dispute',
 'ptype_voidtrip',
 'Duration']

In [29]:
categoricalColumns = ["VendorID_1_0", "store_and_fwd_flag_1_0", "RateCode_stdrate", "RateCode_JFK", "RateCode_Newark", "RateCode_Nassau_W", "RateCode_Negotiated", "RateCode_Group", "Pickup_Manhattan", "DropOff_Brooklyn", "DropOff_Queens", "DropOff_Bronx", "Rain", "Fog", "Snow", "Holiday", "ptype_CCard", "ptype_cash", "ptype_nocharge", "ptype_dispute", "ptype_voidtrip"]

In [30]:
stages = []

In [31]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [ ]:
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]

In [ ]:
features = [""]

In [34]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(samdf)

IllegalArgumentException: 'Field "features" does not exist.\nAvailable fields: VendorID_1_0, store_and_fwd_flag_1_0, RateCode_stdrate, RateCode_JFK, RateCode_Newark, RateCode_Nassau_W, RateCode_Negotiated, RateCode_Group, Pickup_Manhattan, Pickup_Brooklyn, Pickup_Queens, Pickup_Bronx, DropOff_Manhattan, DropOff_Brooklyn, DropOff_Queens, DropOff_Bronx, avg_temp_C, Rain, Fog, Snow, Holiday, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, pickup_zip, pickup_neighborhood, dropoff_zip, dropoff_neighborhood, pickup_dayofweek, pickup_weekday_name, pickup_hour, pickup_month, ptype_CCard, ptype_cash, ptype_nocharge, ptype_dispute, ptype_voidtrip, Duration'

In [41]:
va = VectorAssembler(inputCols = ["VendorID_1_0", "RateCode_stdrate", "RateCode_JFK", "RateCode_Newark", "RateCode_Nassau_W",
                                  "RateCode_Negotiated", "RateCode_Group", "Pickup_Manhattan", "Pickup_Brooklyn", "Pickup_Queens",
                                 "Pickup_Bronx", "DropOff_Manhattan", "DropOff_Brooklyn", "DropOff_Queens", "DropOff_Bronx",
                                 "avg_temp_C", "Rain", "Fog", "Snow", "Holiday",
                                 "passenger_count", "trip_distance", "pickup_longitude", "pickup_latitude", "dropoff_longitude",
                                 "dropoff_latitude", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
                                  "pickup_zip", "dropoff_zip", "pickup_dayofweek",
                                  "pickup_hour", "pickup_month", "ptype_CCard", "ptype_cash", "ptype_nocharge",
                                 "ptype_dispute", "ptype_voidtrip", "Duration"], outputCol = "features")

In [22]:
rf = RandomForestRegressor(featuresCol="features", labelCol = "total_amount")

In [23]:
pipeline = Pipeline(stages=[va, rf])

In [24]:
model = pipeline.fit(samdf)

In [9]:
predictions = model.transform(samdf)

In [15]:
predictions.select("prediction", "total_amount", "features").show(5)

+------------------+------------+--------------------+
|        prediction|total_amount|            features|
+------------------+------------+--------------------+
| 32.11061063363133|         0.4|(42,[5,8,12,15,20...|
|18.828136352096855|        12.6|(42,[5,7,11,15,20...|
| 32.15116127592385|        60.8|(42,[5,7,11,15,20...|
| 10.56338646965404|         8.8|(42,[1,10,14,15,2...|
| 37.67932337841857|       38.63|(42,[1,9,14,15,20...|
+------------------+------------+--------------------+
only showing top 5 rows



In [14]:
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")

In [16]:
rmse = evaluator.evaluate(predictions)

In [17]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.1609


In [25]:
predictions = model.transform(valdf)

In [26]:
predictions.select("prediction", "total_amount", "features").show(5)

+------------------+------------+--------------------+
|        prediction|total_amount|            features|
+------------------+------------+--------------------+
| 25.43383983007029|        23.8|(42,[1,10,11,15,1...|
| 45.67083095191539|       58.01|(42,[1,9,14,15,20...|
| 22.02537810502249|        20.8|(42,[1,9,13,15,20...|
|   8.2726896035792|         3.8|(42,[1,9,13,15,20...|
|24.207286933189703|        24.8|(42,[1,9,13,15,20...|
+------------------+------------+--------------------+
only showing top 5 rows



In [27]:
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")

In [28]:
rmse = evaluator.evaluate(predictions)

In [29]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 1.95625


In [34]:
rfModel = model.stages[1]

In [35]:
print(rfModel)

RandomForestRegressionModel (uid=RandomForestRegressor_491590fd893f56de135b) with 20 trees


In [42]:
gbt = GBTRegressor(featuresCol="features", labelCol = "total_amount", maxIter=10)

In [43]:
pipeline1 = Pipeline(stages=[va, gbt])

In [44]:
modelgbt = pipeline1.fit(samdf)

In [46]:
predictions1 = modelgbt.transform(samdf)

In [47]:
predictions1.select("prediction", "total_amount", "features").show(5)

+------------------+------------+--------------------+
|        prediction|total_amount|            features|
+------------------+------------+--------------------+
| 15.42675204723412|         0.4|(42,[5,8,12,15,20...|
|22.842344393699204|        12.6|(42,[5,7,11,15,20...|
| 63.11846205881489|        60.8|(42,[5,7,11,15,20...|
|  8.86524892366539|         8.8|(42,[1,10,14,15,2...|
| 37.49892390286902|       38.63|(42,[1,9,14,15,20...|
+------------------+------------+--------------------+
only showing top 5 rows



In [48]:
evaluator1 = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")

In [50]:
rmse1 = evaluator1.evaluate(predictions)

In [51]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse1)

Root Mean Squared Error (RMSE) on test data = 1.93234


In [52]:
predictionsnew = modelgbt.transform(valdf)

In [53]:
predictionsnew.select("prediction", "total_amount", "features").show(5)

+------------------+------------+--------------------+
|        prediction|total_amount|            features|
+------------------+------------+--------------------+
|19.532706760687876|        23.8|(42,[1,10,11,15,1...|
|48.571071215247706|       58.01|(42,[1,9,14,15,20...|
|21.045100371653312|        20.8|(42,[1,9,13,15,20...|
| 5.240765631444649|         3.8|(42,[1,9,13,15,20...|
|31.081552871242828|        24.8|(42,[1,9,13,15,20...|
+------------------+------------+--------------------+
only showing top 5 rows



In [54]:
evaluatorgbt = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")

In [55]:
rmse = evaluator.evaluate(predictionsnew)

In [56]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.04745
